In [1]:
!pip install dash==1.19.0  
!pip install jupyter_dash 
!pip install --upgrade plotly

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = JupyterDash(__name__)

In [5]:

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                            ],
                                            value='ALL'
                                            ),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                # Function decorator to specify function input and output
                                
                                
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                               min = 0,
                                               max = 10000,
                                               step = 1000,
                                               value = [min_payload, max_payload],
                                               marks = {
                                                   0: '0 kg',
                                                   2500: '2500',
                                                   5000: '5000',
                                                   7500: '7500',
                                                   10000: '10000'
                                               }), 

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site_value):
    filtered_df = spacex_df
    if entered_site_value == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', 
                     title='Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site_value].groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        title = "Total Success Launches for site {}".format(entered_site_value)
        fig = px.pie(filtered_df, values = 'class count', names = 'class', title = title)
        return fig
        # return the outcomes piechart for a selected site
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
Output(component_id='success-payload-scatter-chart', component_property='figure'),
[Input(component_id='site-dropdown', component_property='value'),
 Input(component_id='payload-slider', component_property='value')]
)
def get_scatter(value1, value2):
    filtered_df2_1=spacex_df[(spacex_df['Payload Mass (kg)'] > value2[0]) & (spacex_df['Payload Mass (kg)'] < value2[1])]
    
    if value1 == 'ALL':
        fig= px.scatter(filtered_df2_1,x="Payload Mass (kg)",y="class",color="Booster Version Category",\
        title="Correlation between Payload and Success for All sites")
        return fig
    else:
        filtered_df2_2=filtered_df2_1[filtered_df2_1['Launch Site']==value1]
        fig= px.scatter(filtered_df2_2,x="Payload Mass (kg)",y="class",color="Booster Version Category",\
        title=f"Correlation between Payload and Success for site {value1}")
        return fig
    
# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

Later in browser use the url

http://localhost:8090

In [15]:
spacex_df

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.00,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.00,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.00,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.00,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.00,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,Success,0,3170.00,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,Success,0,3325.00,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,Success,0,2296.00,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,Success,0,1316.00,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,Success,0,4535.00,F9 v1.1,v1.1
